## Packages

In [1]:
import cv2
import numpy as np

## Loading YOLO

In [2]:
yolo_model = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

classes = []

with open("coco_classes.txt", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = yolo_model.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_model.getUnconnectedOutLayers()]

colors = np.random.uniform(0, 255, size=len(classes))

## Loading An Image

In [3]:
img = cv2.imread("images/0030.jpg")
height, width, channels = img.shape

## Detecting Objects

In [4]:
blob = cv2.dnn.blobFromImage(img, 0.004, (416,416), (0,0,0), True, crop=False)
yolo_model.setInput(blob)
outs = yolo_model.forward(output_layers)

## Finding Bounding Boxes Coordinates

In [5]:
boxes = []
confidences = []
class_ids = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if confidence > 0.5:  # object dedected
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)

            # Rectangle Coordinates

            x_top_left = int(center_x - w/2)
            y_top_left = int(center_y - h/2)

            boxes.append([x_top_left, y_top_left, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

## Non-max Suppression

In [6]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
number_of_objects_detected = len(boxes)
font = cv2.FONT_HERSHEY_COMPLEX
for i in range(number_of_objects_detected):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 3)
        cv2.putText(img, label, (x, y-5), font, 0.7, color, 2)

In [8]:
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()